https://nixtlaverse.nixtla.io/neuralforecast/docs/tutorials/longhorizon_transformers.html

In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon

from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, FEDformer, PatchTST

from matplotlib import pyplot as plt

In [3]:
data_path = "/Users/pepijnschouten/Desktop/Python_Scripts/" \
    "Python_scripts_Varia/Deep_Learning/NIXTLA/data"

Y_df, _, _ = LongHorizon.load(directory=data_path, group='ETTm2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.2 * n_time)

print(Y_df.groupby('unique_id').head(2))

       unique_id                  ds         y
0           HUFL 2016-07-01 00:00:00 -0.041413
1           HUFL 2016-07-01 00:15:00 -0.185467
57600       HULL 2016-07-01 00:00:00  0.040104
57601       HULL 2016-07-01 00:15:00 -0.214450
115200      LUFL 2016-07-01 00:00:00  0.695804
115201      LUFL 2016-07-01 00:15:00  0.434685
172800      LULL 2016-07-01 00:00:00  0.434430
172801      LULL 2016-07-01 00:15:00  0.428168
230400      MUFL 2016-07-01 00:00:00 -0.599211
230401      MUFL 2016-07-01 00:15:00 -0.658068
288000      MULL 2016-07-01 00:00:00 -0.393536
288001      MULL 2016-07-01 00:15:00 -0.659338
345600        OT 2016-07-01 00:00:00  1.018032
345601        OT 2016-07-01 00:15:00  0.980124


Model definitions and training

In [5]:
horizon = 4 * 14
models = [Informer(
    h=horizon,                 
    input_size=horizon,           
    max_steps=1000,               
    val_check_steps=100,          
    early_stop_patience_steps=3,
    gpu=False,
    ),
          Autoformer(
    h=horizon,
    input_size=horizon,
    max_steps=1000,
    val_check_steps=100,
    early_stop_patience_steps=3,
    gpu=False,
    ),
          PatchTST(
    h=horizon,
    input_size=horizon,
    max_steps=1000,
    val_check_steps=100,
    early_stop_patience_steps=3,
    gpu=False,
    ),
]

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [6]:
nf = NeuralForecast(
    models=models,
    freq='15min',
)

Y_hat_df = nf.cross_validation(
    df=Y_df,
    val_size=val_size,
    test_size=test_size,
    n_windows=None,
    
)

print(Y_hat_df.head())

TypeError: Trainer.__init__() got an unexpected keyword argument 'gpu'

Evaluate the results

In [ ]:
Y_hat_df = Y_hat_df.reset_index()
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')
plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')
plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')
plt.plot(Y_plot['ds'], Y_plot['PatchTST'], label='PatchTST')
plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()
plt.show()

